In [ ]:
def generate_metric_table(df, athlete_id, selected_metrics, unit_dict, pdf, custom_labels):
    """
    Generate a visually enhanced table summarizing selected metrics with Onboarding, Most Recent, PR values, and dates/phases.

    Args:
        df (pd.DataFrame): The dataset containing test data.
        athlete_id (int): Athlete ID.
        selected_metrics (list): List of tuples specifying metrics to include.
        unit_dict (dict): Dictionary mapping (Test Type, Test Sub-Type) to units.
        pdf (PdfPages): PDF object to save the plot.
        custom_labels (list): List of custom labels corresponding to selected metrics.

    Returns:
        None
    """
    # Filter data for the specific athlete
    athlete_data = df[df['ID'] == athlete_id]
    if athlete_data.empty:
        print(f"No data found for athlete ID {athlete_id}.")
        return

    # Initialize table data
    table_data = []
    column_headers = ["Metric", "Onboarding", "Onboarding Date/Phase", 
                      "Most Recent", "Most Recent Date/Phase", 
                      "PR", "PR Date/Phase"]

    for idx, (test_type, test_sub_type, metric) in enumerate(selected_metrics):
        # Use custom label for the metric
        metric_label = custom_labels[idx]

        # Filter data for the specific metric
        metric_data = athlete_data[
            (athlete_data['Test Type'] == test_type) &
            (athlete_data['Test Sub-Type'] == test_sub_type)
        ]

        if metric_data.empty:
            onboarding_value = "N/A"
            onboarding_date_phase = "N/A"
            most_recent_value = "N/A"
            most_recent_date_phase = "N/A"
            pr_value = "N/A"
            pr_date_phase = "N/A"
        else:
            # Convert metric column to numeric to handle mixed types
            if metric in metric_data.columns:
                metric_data = metric_data.copy()
                metric_data[metric] = pd.to_numeric(metric_data[metric], errors='coerce')

            # Onboarding value (Test Number = 1)
            onboarding_data = metric_data[metric_data['Test Number'] == 1]
            onboarding_value = (
                onboarding_data[metric].iloc[0]
                if not onboarding_data.empty and metric in onboarding_data.columns
                else "N/A"
            )
            onboarding_date_phase = (
                f"{onboarding_data['Date'].iloc[0]}\n{onboarding_data['Phase'].iloc[0]}"
                if not onboarding_data.empty and 'Date' in onboarding_data.columns and 'Phase' in onboarding_data.columns
                and pd.notna(onboarding_data['Phase'].iloc[0])
                else onboarding_data['Date'].iloc[0]
                if not onboarding_data.empty and 'Date' in onboarding_data.columns
                else "N/A"
            )

            # Most recent value (highest Test Number)
            most_recent_data = metric_data[metric_data['Test Number'] == metric_data['Test Number'].max()]
            most_recent_value = (
                most_recent_data[metric].iloc[0]
                if not most_recent_data.empty and metric in most_recent_data.columns
                else "N/A"
            )
            most_recent_date_phase = (
                f"{most_recent_data['Date'].iloc[0]}\n{most_recent_data['Phase'].iloc[0]}"
                if not most_recent_data.empty and 'Date' in most_recent_data.columns and 'Phase' in most_recent_data.columns
                and pd.notna(most_recent_data['Phase'].iloc[0])
                else most_recent_data['Date'].iloc[0]
                if not most_recent_data.empty and 'Date' in most_recent_data.columns
                else "N/A"
            )

            # PR value (highest value in the metric column)
            pr_value = (
                metric_data[metric].max()
                if metric in metric_data.columns and not metric_data[metric].isna().all()
                else "N/A"
            )
            pr_data = metric_data[metric_data[metric] == pr_value]
            pr_date_phase = (
                f"{pr_data['Date'].iloc[0]}\n{pr_data['Phase'].iloc[0]}"
                if not pr_data.empty and 'Date' in pr_data.columns and 'Phase' in pr_data.columns
                and pd.notna(pr_data['Phase'].iloc[0])
                else pr_data['Date'].iloc[0]
                if not pr_data.empty and 'Date' in pr_data.columns
                else "N/A"
            )

        # Get units for the metric
        units = unit_dict.get((test_type, test_sub_type), '')

        # Format values with units
        def format_with_units(value):
            return f"{value} {units}" if value != "N/A" and units else value

        onboarding_value = format_with_units(onboarding_value)
        most_recent_value = format_with_units(most_recent_value)
        pr_value = format_with_units(pr_value)

        # Append row to table data
        table_data.append([
            metric_label,
            onboarding_value, onboarding_date_phase,
            most_recent_value, most_recent_date_phase,
            pr_value, pr_date_phase
        ])

    # Create the table plot
    fig, ax = plt.subplots(figsize=(14, len(table_data) * 0.6 + 2))  # Adjust height dynamically based on rows
    ax.axis('off')  # Remove axes

    # Add table to the figure
    table = ax.table(
        cellText=table_data,
        colLabels=column_headers,
        loc='center',
        cellLoc='center',
        colColours=['#4f81bd'] * len(column_headers),  # Blue header background
        bbox=[0, 0, 1, 1]  # Cover the entire figure
    )
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.auto_set_column_width(col=list(range(len(column_headers))))

    # Bold headers
    for (row, col), cell in table.get_celld().items():
        if row == 0:  # Header row
            cell.set_text_props(weight='bold', color='white', fontsize=9)

    # Alternating row colors for readability
    for row, col in table.get_celld():
        if row > 0:  # Skip header row
            if row % 2 == 0:
                table.get_celld()[(row, col)].set_facecolor("#f2f2f2")  # Light gray
            else:
                table.get_celld()[(row, col)].set_facecolor("white")  # White

    # Highlight metric values for emphasis
    for row, col in table.get_celld():
        if col in [1, 3, 5] and row > 0:  # Metric value columns
            table.get_celld()[(row, col)].set_text_props(weight='bold', fontsize=11)

    # Add title
    ax.set_title(
        "Metric Summary",
        fontsize=16,
        fontweight='bold',
        loc='center',
        pad=20
    )

    # Save the table to the PDF
    pdf.savefig(fig)
    plt.close(fig)

